In [3]:
import os
import sys
import json
from pathlib import Path

import torch
import numpy as np

In [91]:
def training_handling_challenges(data_folder: str):

    
    file_path = Path(data_folder).expanduser()
    with file_path.open("r") as f:
        file_dict = json.load(f)

    for data in file_dict:
        train_data = file_dict[data]['train']
        print(data)
        for train_item in train_data:
            print(train_item['input'])
        


def training_handling_solutions(data_folder: str):
    file_path = Path(data_folder).expanduser()
    with file_path.open("r") as f:
        file_dict = json.load(f)
        print(file_dict['007bbfb7'])
        
        
        



training_handling_challenges(f"~/ARC-AGI-Model/src/data_pipeline/arc-data-cleaned/arc-agi_training_challenges.json")






007bbfb7
[[0, 7, 7], [7, 7, 7], [0, 7, 7]]
[[4, 0, 4], [0, 0, 0], [0, 4, 0]]
[[0, 0, 0], [0, 0, 2], [2, 0, 2]]
[[6, 6, 0], [6, 0, 0], [0, 6, 6]]
[[2, 2, 2], [0, 0, 0], [0, 2, 2]]
00d62c1b
[[0, 0, 0, 0, 0, 0], [0, 0, 3, 0, 0, 0], [0, 3, 0, 3, 0, 0], [0, 0, 3, 0, 3, 0], [0, 0, 0, 3, 0, 0], [0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 3, 0, 3, 0, 0, 0, 0, 0], [0, 0, 0, 3, 0, 3, 0, 0, 0, 0], [0, 0, 3, 0, 0, 0, 3, 0, 0, 0], [0, 0, 0, 0, 0, 3, 0, 3, 0, 0], [0, 0, 0, 3, 0, 3, 3, 0, 0, 0], [0, 0, 3, 3, 3, 0, 0, 0, 0, 0], [0, 0, 0, 3, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 3, 0, 0, 0, 0], [0, 0, 0, 0, 3, 0, 0, 0, 0, 0], [0, 3, 3, 0, 3, 3, 0, 3, 0, 0], [3, 0, 0, 3, 0, 0, 3, 0, 3, 0], [0, 0, 0, 3, 0, 0, 3, 3, 0, 0], [0, 0, 0, 3, 0, 0, 3, 0, 0, 0], [0, 0, 0, 3, 0, 0, 3, 0, 0, 0], [0, 0, 0, 0, 3, 3, 0, 3, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 3, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 3, 3, 3, 